In [1]:

import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
import pandas as pd
from scipy.optimize import minimize

In [2]:
df = pd.read_csv("investment_information.csv")

In [3]:
df

,wlmt,amazon,mcrsft,chevron
0,141.789993,84.000000,239.820007,179.490005
1,143.869995,103.129997,247.809998,174.020004
2,142.130005,94.230003,249.419998,160.770004
3,147.449997,103.290001,288.299988,163.160004
4,150.970001,105.449997,307.260010,168.580002
5,146.869995,120.580002,328.390015,150.619995
6,157.179993,130.360001,340.540009,157.350006
7,159.860001,133.679993,335.920013,163.660004
8,162.610001,138.009995,327.760010,161.100006
9,159.929993,127.120003,315.750000,168.619995


In [18]:
# month over month returns, correlation, covariance matrix

In [6]:
def momr(col):
    arr_1 = make_array()
    k = 0
    while k <= (len(col)-2):
        val = (col[k+1] - col[k])/col[k]
        arr_1 = np.append(arr_1, val)
        k += 1
    return arr_1

In [12]:
def momr_total(df):
    i = 0 
    new_data = pd.DataFrame()
    col_num = df.shape[1]
    while i <= (col_num-1):
        example = momr(df.iloc[:, i])
        new_data[df.columns[i]] = example
        i += 1
    return new_data*100, df.corr(), df.cov()

In [23]:

momr_total(df)[0], momr_total(df)[1], momr_total(df)[2]

(        wlmt     amazon     mcrsft    chevron
 0   1.466960  22.773806   3.331662  -3.047524
 1  -1.209418  -8.629879   0.649691  -7.614067
 2   3.743046   9.614770  15.588161   1.486596
 3   2.387253   2.091196   6.576491   3.321891
 4  -2.715775  14.348037   6.876913 -10.653700
 5   7.019812   8.110797   3.699867   4.468206
 6   1.705057   2.546787  -1.356668   4.010167
 7   1.720255   3.239080  -2.429151  -1.564217
 8  -1.648120  -7.890727  -3.664269   4.667901
 9   2.175959   4.696344   7.081547 -13.574902
 10  0.312095   5.642806   6.678305  -2.600696
 11  0.461022   0.149289   0.842832   0.366355,
              wlmt    amazon    mcrsft   chevron
 wlmt     1.000000  0.939311  0.885850 -0.629537
 amazon   0.939311  1.000000  0.934842 -0.718798
 mcrsft   0.885850  0.934842  1.000000 -0.786369
 chevron -0.629537 -0.718798 -0.786369  1.000000,
                wlmt      amazon       mcrsft     chevron
 wlmt      79.137164  160.582491   333.291731  -66.293215
 amazon   160.582491  369.

In [19]:
# solving and creating lp model

In [37]:
def objective_func(x):
    return np.dot(np.dot(np.transpose(x),momr_total(df)[2]),x)

In [38]:
def constraint_1(x):
    return sum(x)-1

In [39]:
def constraint_2(x):
    R = [np.mean(momr_total(df)[0].iloc[:, 0]), np.mean(momr_total(df)[0].iloc[:, 1]),
        np.mean(momr_total(df)[0].iloc[:, 2]), np.mean(momr_total(df)[0].iloc[:, 3])]
    Rm = .05
    return np.dot(np.transpose(R), x) - Rm

In [40]:
b = (0, None)

In [41]:
bnds = (b, b, b, b)

In [42]:
con1 = {'type':'eq','fun':constraint_1}
con2 = {'type':'ineq','fun':constraint_2}
cons = [con1, con2]

In [43]:
X = [.1, .3, .2, .4]

In [44]:
sol = minimize(objective_func, X,  method='SLSQP', bounds=bnds, constraints=cons)

In [45]:
sol


     fun: 19.030278703693906
     jac: array([ 39.05545259,  55.47911811,  70.62585402,  36.62818933])
 message: 'Optimization terminated successfully'
    nfev: 20
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 0.59011701,  0.        ,  0.        ,  0.40988299])